# Implementing KMeans

In this exercise we will be implementing the k-means clustering algorithm. For an introduction on how this algorithm works I recommend you to read:
- [K-Means Clustering Algorithm Overview](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html)

The following figures illustrate the steps the algorithm follows to find two centroids (taken from the previous link):

![K-Means algorithm](http://bigdata.cesga.es/files/kmeansViz.png)

## Dependencies

In [1]:
from __future__ import print_function
import math
from collections import namedtuple

## Parameters

In [2]:
# Number of clusters to find
K = 5
# Convergence threshold
THRESHOLD = 0.1
# Maximum number of iterations
MAX_ITERS = 20

## Load data

In [3]:
def parse_coordinates(line):
    fields = line.split(',')
    return (float(fields[3]), float(fields[4]))

In [4]:
data = sc.textFile('datasets/locations')

In [5]:
points = data.map(parse_coordinates)

## Useful functions

In [6]:
def distance(p1, p2):  
    "Returns the squared distance between two given points"
    return (p1[0] - p2[0])** 2 + (p1[1] - p2[1])** 2

def closest_centroid(point, centroids):    
    "Returns the index of the closest centroid to the given point: eg. the cluster this point belongs to"
    distances = [distance(point, c) for c in centroids]
    shortest = min(distances)
    return distances.index(shortest)

def add_points(p1, p2):
    "Returns the sum of two points"
    return (p1[0] + p2[0], p1[1] + p2[1])

## Iteratively calculate the centroids

In [7]:
%%time
# Initial centroids: we just take K randomly selected points
centroids = points.takeSample(False, K, 42)

# The first iteration should always run
variation = THRESHOLD + 1
iteration = 0

while variation > THRESHOLD  and iteration < MAX_ITERS:
     # Map each point to (centroid, (point, 1))
    with_centroids = points.map(lambda p: (closest_centroid(p, centroids), (p, 1)))
    # For each centroid reduceByKey adding the coordinates of all the points
    # and keeping track of the number of points
    cluster_stats = with_centroids.reduceByKey(lambda (p1, n1), (p2, n2):  (add_points(p1, p2), n1 + n2))
    # For each existing centroid find the new centroid location calculating the average of each closest point
    new_centroids = cluster_stats.map(lambda (c, ((x, y), n)): (c, (x/n, y/n))).collect()
    # Calculate the variation between old and new centroids
    variation = 0
    for (old_centroid_id, new_centroid) in new_centroids: 
        variation += distance(centroids[old_centroid_id], new_centroid)
    print('Variation in iteration {}: {}'.format(iteration, variation))
    # Replace old centroids with the new values
    for (old_centroid_id, new_centroid) in new_centroids: 
        centroids[old_centroid_id] = new_centroid
    iteration += 1
        
print('Final centroids: {}'.format(centroids))

Variation in iteration 0: 4989.32008451
Variation in iteration 1: 2081.17551268
Variation in iteration 2: 1.6011620119
Variation in iteration 3: 2.55059475168
Variation in iteration 4: 0.994848416636
Variation in iteration 5: 0.0381850235415
Final centroids: [(35.08592000544936, -112.57643826547803), (0.0, 0.0), (38.05200414101911, -121.20324355675143), (43.891507710205694, -121.32350131512835), (34.28939789970032, -117.77840744773651)]
CPU times: user 84.9 ms, sys: 9.17 ms, total: 94.1 ms
Wall time: 21 s


## Reviewing the steps we have done

We start with lines of text:

In [8]:
line = data.first()
print(line)

2017-03-15:10:10:20,Motorola F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253


We have to convert them to points:

In [9]:
parse_coordinates(line)

(33.6894754264, -117.543308253)

In [10]:
points = data.map(parse_coordinates)
point = points.first()
print(point)

(33.6894754264, -117.543308253)


We select some arbitrary points from our RDD of points as the initial centroids:

In [11]:
# Initial centroids: we just take K randomly selected points
centroids = points.takeSample(False, K, 42)
print(centroids)

[(33.4898547489, -111.63617776), (33.5505811202, -111.243243255), (36.5697673035, -120.79623245), (37.7152004069, -121.473355818), (34.3743073814, -117.184154207)]


NOTE: Using the closest_centroid() funtion, we are able so calculate the **index of the closest centroid** to a given point:

In [12]:
closest_centroid_index = closest_centroid(point, centroids)
print(closest_centroid_index)

4


In [13]:
centroids[closest_centroid_index]

(34.3743073814, -117.184154207)

#### STEP 1: Assign a point to its centroid
**point -> (closest_centroid_id, (point, 1))**

In [14]:
with_centroids = points.map(lambda p: (closest_centroid(p, centroids), (p, 1)))
print(with_centroids.first())

(4, ((33.6894754264, -117.543308253), 1))


#### STEP 2: Preparation to calculate new centroids
**(closest_centroid_id, (point, 1)) -> (closest_centroid_id, (sum(points), total_number_of_points)**

In [15]:
cluster_stats = with_centroids.reduceByKey(lambda (p1, n1), (p2, n2):  (add_points(p1, p2), n1 + n2))
print(cluster_stats.first())

(0, ((841812.50940875, -2768072.601871161), 24640))


#### STEP 3: We calculate the new centroids
**closest_centroid_id, (sum(points), total_number_of_points) -> (centroid_id, new_centroid)** 

where `new_centroid = (sum_x/total, sum_y/total)`.

In [16]:
new_centroids = cluster_stats.map(lambda (c, ((x, y), n)): (c, (x/n, y/n))).collect()
print(new_centroids[0])

(0, (34.164468726004465, -112.34060884217374))


#### STEP 4: We calculate the variation
Finally we just calculate the variation between old and new centroids to verify if we have to continue iterating.

In [17]:
variation = 0
for (old_centroid_id, new_centroid) in new_centroids:
    variation += distance(centroids[old_centroid_id], new_centroid)
print(variation)

4989.32008451
